### 数据处理说明
#### 数据来源：2009年微波辐射计亮温数据和探空数据
#### 微波辐射计数据
- 08时Wyoming数据对应08:00~08:30微波辐射计数据平均值

In [1]:
import pandas as pd
import os
import calendar

# 自定义函数
import search_filepth_module

In [14]:
Year = '2009'

In [15]:
# 获取特定年份全年的所有日期
def getMothDate(year, month):
    """
    返回某年某月的所有日期
    :param year:
    :param month:
    :return:
    """
    date_list = []
    for i in range(calendar.monthrange(year, month)[1] + 1)[1:]:
        str1 = str(year) +'/'+ str("%02d" % month) +'/'+ str("%02d" % i)
        date_list.append(str1)
    return date_list
 
list = []
year = 2009
for month in range(1,13):
    date_list = getMothDate(year, month)
    for date in date_list:
        list.append(date)
# 生成纵坐标为全年日期，横坐标为亮温数据和探空数据的DataFrame表格
statistics = pd.DataFrame(columns=['lv1_08','Temperature_08','Relative_humidity_08','lv1_20','Temperature_20','Relative_humidity_20'],index=list)

## 微波辐射计数据

In [16]:
microwave_dirpath = r'I:\Data\Personal Data\graduation project\SACOL\microwave\\' + Year
microwave_target_str = 'lv1'
microwave_filepaths = search_filepth_module.search_filepath(microwave_dirpath,microwave_target_str)

In [17]:
# 创建用于存储08和20亮温数据的DataFrame
channel = ['10', 'Tamb(K)', 'Rh(%)', 'Pres(mb)', 'Tir(K)', 'Rain', 'Azim', 'Elev', 'TkBB(K)', ' 22.235', ' 23.035', ' 23.835', ' 26.235', ' 30.000', ' 51.250', ' 52.280', ' 53.850', ' 54.940', ' 56.660', ' 57.290', ' 58.800']
lv1_08 = pd.DataFrame(index=list,columns=channel)
lv1_20 = pd.DataFrame(index=list,columns=channel)

In [18]:
for microwave_filepath in microwave_filepaths:
    lv1_data = pd.read_csv(microwave_filepath,index_col='Record')
    lv1_data.drop(['Unnamed: 23'],axis=1,inplace=True)
    # print(microwave_filepath)
    # print(microwave_filepath[70:80].replace('-','/'),lv1_data.shape)

    num_08 = 0
    num_20 = 0
    flag_08 = 0
    flag_20 = 0
    for i in range(lv1_data.shape[0]):
        # print(lv1_data.loc[i+1,'Date/Time'])
        point = lv1_data.loc[i+1,'Date/Time']
        if((point[9:14] >= '08:00') and (point[9:14] <= '08:30')):
            num_08 += 1
            if(flag_08 == 0):
                flag_08 = i+1
        if((point[9:14] >= '20:00') and (point[9:14] <= '20:30')):
            num_20 += 1
            if(flag_20 == 0):
                flag_20 = i+1
    statistics.loc[microwave_filepath[70:80].replace('-','/'),'lv1_08'] = num_08
    statistics.loc[microwave_filepath[70:80].replace('-','/'),'lv1_20'] = num_20
    if(flag_08 != 0):
        lv1_08.loc[microwave_filepath[70:80].replace('-','/'),] = lv1_data.iloc[flag_08-1:flag_08+num_08-1,].mean()
    if(flag_20 != 0):
        lv1_20.loc[microwave_filepath[70:80].replace('-','/'),] = lv1_data.iloc[flag_20-1:flag_20+num_20-1,].mean()

## 探空数据

In [19]:
Wyoming_dirpath = r'I:\Data\Personal Data\graduation project\yuzhongzhan\FSL1'
Wyoming_target_str = Year
Wyoming_filepaths = []
for root,dirs,files in os.walk(Wyoming_dirpath):
    for file in files:
        # print(file[30:34])
        if(Wyoming_target_str == file[30:34]):
            Wyoming_filepath = os.path.join(root,file)
            Wyoming_filepaths.append(Wyoming_filepath)

In [20]:
height = ['0.00','0.10','0.20','0.30','0.40','0.50','0.60','0.70','0.80','0.90','1.00','1.25','1.50','1.75','2.00','2.25','2.50','2.75','3.00','3.25','3.50','3.75','4.00','4.25','4.50','4.75','5.00','5.25','5.50','5.75','6.00','6.25','6.50','6.75','7.00','7.25','7.50','7.75','8.00','8.25','8.50','8.75','9.00','9.25','9.50','9.75','10.00']
Temperature_08 = pd.DataFrame(index=list,columns=height)
Temperature_20 = pd.DataFrame(index=list,columns=height)
Relative_humidity_08 = pd.DataFrame(index=list,columns=height)
Relative_humidity_20 = pd.DataFrame(index=list,columns=height)

In [21]:
for Wyoming_filepath in Wyoming_filepaths:
    head_col = ['Press','Altitude','Temperature','Relative_humidity','NaN1','NaN2','NaN3']
    Wyoming_data = pd.read_table(Wyoming_filepath,sep='\s+',skiprows=6,names=head_col)
    Wyoming_data.index = height
    # Wyoming_data['Altitude'] = Wyoming_data['Altitude']-1965
    Wyoming_date = Wyoming_filepath[88:92]+'/'+Wyoming_filepath[92:94]+'/'+Wyoming_filepath[94:96]
    print(Wyoming_date)
    if(Wyoming_filepath[96:98] == '00'):
        # 统计可用数据个数
        statistics.loc[Wyoming_date,'Temperature_08'] = Wyoming_data.loc[Wyoming_data['Temperature']>-100].Temperature.count()
        # 集中数据
        Temperature_08.loc[Wyoming_date,] = Wyoming_data['Temperature']
        # 统计可用数据个数
        statistics.loc[Wyoming_date,'Relative_humidity_08'] = Wyoming_data.loc[Wyoming_data['Relative_humidity']>-100].Temperature.count()
        # 集中数据
        Relative_humidity_08.loc[Wyoming_date,] = Wyoming_data['Relative_humidity']
    elif(Wyoming_filepath[96:98] == '12'):
        # 统计可用数据个数
        statistics.loc[Wyoming_date,'Temperature_20'] = Wyoming_data.loc[Wyoming_data['Temperature']>-100].Temperature.count()
        # 集中数据
        Temperature_20.loc[Wyoming_date,] = Wyoming_data['Temperature']
        # 统计可用数据个数
        statistics.loc[Wyoming_date,'Relative_humidity_20'] = Wyoming_data.loc[Wyoming_data['Relative_humidity']>-100].Temperature.count()
        # 集中数据
        Relative_humidity_20.loc[Wyoming_date,] = Wyoming_data['Relative_humidity']

2009/01/01
2009/01/01
2009/01/02
2009/01/02
2009/01/03
2009/01/03
2009/01/04
2009/01/04
2009/01/05
2009/01/05
2009/01/06
2009/01/06
2009/01/07
2009/01/07
2009/01/08
2009/01/08
2009/01/09
2009/01/09
2009/01/10
2009/01/10
2009/01/11
2009/01/11
2009/01/12
2009/01/12
2009/01/13
2009/01/13
2009/01/14
2009/01/14
2009/01/15
2009/01/15
2009/01/16
2009/01/16
2009/01/17
2009/01/17
2009/01/18
2009/01/18
2009/01/19
2009/01/19
2009/01/20
2009/01/20
2009/01/21
2009/01/21
2009/01/22
2009/01/22
2009/01/23
2009/01/23
2009/01/24
2009/01/24
2009/01/25
2009/01/25
2009/01/26
2009/01/26
2009/01/27
2009/01/27
2009/01/28
2009/01/28
2009/01/29
2009/01/29
2009/01/30
2009/01/30
2009/01/31
2009/01/31
2009/02/01
2009/02/01
2009/02/02
2009/02/02
2009/02/03
2009/02/03
2009/02/04
2009/02/04
2009/02/05
2009/02/05
2009/02/06
2009/02/06
2009/02/07
2009/02/07
2009/02/08
2009/02/08
2009/02/09
2009/02/09
2009/02/10
2009/02/10
2009/02/11
2009/02/11
2009/02/12
2009/02/12
2009/02/13
2009/02/13
2009/02/14
2009/02/14
2009/02/15

In [23]:
# 储存数据
statistics.to_csv(r'I:\Data\Personal Data\graduation project\Code\Machine Learning\basicdata\statistics.csv')
lv1_08.to_csv(r'I:\Data\Personal Data\graduation project\Code\Machine Learning\basicdata\lv1_08.csv')
lv1_20.to_csv(r'I:\Data\Personal Data\graduation project\Code\Machine Learning\basicdata\lv1_20.csv')
Temperature_08.to_csv(r'I:\Data\Personal Data\graduation project\Code\Machine Learning\basicdata\Temperature_08.csv')
Temperature_20.to_csv(r'I:\Data\Personal Data\graduation project\Code\Machine Learning\basicdata\Temperature_20.csv')
Relative_humidity_08.to_csv(r'I:\Data\Personal Data\graduation project\Code\Machine Learning\basicdata\Relative_humidity_08.csv')
Relative_humidity_20.to_csv(r'I:\Data\Personal Data\graduation project\Code\Machine Learning\basicdata\Relative_humidity_20.csv')